In [248]:
import pandas as pd
import os

import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2, SelectKBest
import numpy as np
from sklearn.metrics import accuracy_score


In [249]:
dataframes = dict()
for dataset_name in os.listdir('data'):
    if dataset_name.endswith('.csv'):
        dataframes[dataset_name.split(".")[0]] = pd.read_csv('clean_data/' + dataset_name) 

data = pd.merge(dataframes['teams_post'], dataframes['teams'], on=['tmID', 'year'], how='right')
data = pd.merge(data, dataframes['coaches'], on=['tmID', 'year'], suffixes=('_team', '_coach'))
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDWinner', 'year'], suffixes=('_team_post', '_series'), how='left')
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDLoser', 'year'], suffixes=('_team_post', '_series'), how='left')
data = pd.merge(data, dataframes['players_teams'], on=['tmID', 'year'], suffixes=('_team', '_player'))
data = pd.merge(data, dataframes['players'], left_on='playerID', right_on='bioID')
data = pd.merge(data, dataframes['awards_players'], on=['playerID'], how='left')

data.shape
label = "playoff"
df = data
df['confID'].value_counts()

WE    1422
EA    1176
Name: confID, dtype: int64

In [250]:
features = df.columns
toDrop = [label,"rank","semis","finals","firstRound"]
for col in toDrop:
    features = features.drop(col)
assert len(features) < len(df.columns)

X = df[features]  # Features

In [251]:
X

,year_x,tmID,W_team_post,L_team_post,franchID,confID,name,o_fgm,o_fga,o_ftm,...,PostthreeMade,PostDQ,bioID,pos,height,weight,college,birthDate,award,year_y
0,9,ATL,NaN,NaN,ATL,EA,Atlanta Dream,895,2258,542,...,0,0,balesal01w,C,79.0,218,Duke,1985-04-04,NaN,NaN
1,8,IND,3.0,3.0,IND,EA,Indiana Fever,907,2167,478,...,0,0,balesal01w,C,79.0,218,Duke,1985-04-04,NaN,NaN
2,9,IND,1.0,2.0,IND,EA,Indiana Fever,876,2179,495,...,0,0,balesal01w,C,79.0,218,Duke,1985-04-04,NaN,NaN
3,9,ATL,NaN,NaN,ATL,EA,Atlanta Dream,895,2258,542,...,0,0,haynikr01w,G,69.0,147,Michigan State,1983-06-17,NaN,NaN
4,10,DET,3.0,2.0,DET,EA,Detroit Shock,980,2277,539,...,0,0,haynikr01w,G,69.0,147,Michigan State,1983-06-17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593,9,WAS,NaN,NaN,WAS,EA,Washington Mystics,885,2131,435,...,0,0,langhcr01w,F-C,74.0,184,Maryland,1986-10-27,Most Improved Player,10.0
2594,10,WAS,0.0,2.0,WAS,EA,Washington Mystics,933,2205,580,...,0,0,langhcr01w,F-C,74.0,184,Maryland,1986-10-27,Most Improved Player,10.0
2595,9,WAS,NaN,NaN,WAS,EA,Washington Mystics,885,2131,435,...,0,0,vaughkr01w,F,73.0,160,Virginia Commonwealth,1986-09-08,NaN,NaN
2596,9,WAS,NaN,NaN,WAS,EA,Washington Mystics,885,2131,435,...,0,0,vaughkr01w,F,73.0,160,Virginia Commonwealth,1986-09-08,NaN,NaN


In [252]:
df['playoff']
le = LabelEncoder()
y = le.fit_transform(df['playoff'])
for x in df.columns:
    df[x] = le.fit_transform(df[x])   

In [253]:
available_columns = list(features)

key_predictors  = []
features_values = pd.DataFrame(df, columns = available_columns)
target_values   = df["playoff"].values
chi2_results    = chi2(features_values, target_values)
best_chi2_cols  = SelectKBest(chi2, k= 35)

best_chi2_cols.fit(features_values, target_values)

best_chi2_features = features_values.columns[best_chi2_cols.get_support()]

for k in (best_chi2_features):
     key_predictors.append(k)
key_predictors.append("playoff")
key_predictors.append("year_x")
key_predictors.append("confID")
df = df[key_predictors]

In [254]:
clf = DecisionTreeClassifier()

In [255]:
def split_data(df):
    return df[df['confID'] == 0], df[df['confID'] == 1]

In [256]:


def train_model(year, model, data, label):


        train_data = data[data["year_x"] < year ]
        test_data  = data[data["year_x"] == year ]

        X_train, X_test = train_data.drop([label], axis=1), test_data.drop([label], axis=1)
        y_train, y_test = train_data[label], test_data[label]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        return y_test, y_pred
        


def train_evaluate_decision_tree_graph(model, data):

    accuracy_scores = []
    years_tested    = []
    data            = data.sort_values(by="year_x")
    data1, data2    = df[df['confID'] == 0], df[df['confID'] == 1]
    years           = sorted(data["year_x"].unique())

    for year in years[2:]:
        y_test1, y_pred1 = train_model(year, model, data1, label)
        y_test2, y_pred2 = train_model(year, model, data2, label)

        y_test = np.concatenate([y_test1, y_test2])
        y_pred = np.concatenate([y_pred1, y_pred2])

        accuracy = accuracy_score(y_test, y_pred)
        accuracy_scores.append(accuracy)
        years_tested.append(year)

    return years_tested, accuracy_scores

train_evaluate_decision_tree_graph(clf, df)



([2, 3, 4, 5, 6, 7, 8, 9],
 [0.8482758620689655,
  0.8587786259541985,
  0.9446494464944649,
  0.8467741935483871,
  0.9279661016949152,
  0.9873949579831933,
  0.9014598540145985,
  0.872093023255814])